# Food Truck Analysis

In [24]:
import pandas as pd
import altair as alt

In [25]:
truck_df = pd.read_csv('pipeline/data/truck.csv')
payment_df = pd.read_csv('pipeline/data/payment_method.csv')
transaction_df = pd.read_csv('pipeline/data/transaction.csv', parse_dates=['at'])

all_data_df = pd.merge(transaction_df, truck_df, on='truck_id')
all_data_df = pd.merge(all_data_df, payment_df, on='payment_method_id')
all_data_df = all_data_df.drop(columns=['truck_id', 'payment_method_id'])
all_data_df

,transaction_id,total,at,truck_name,truck_description,has_card_reader,fsa_rating,payment_method
0,1,960.0,2025-10-19 09:04:00,Burrito Madness,An authentic taste of Mexico.,1,4,cash
1,2,700.0,2025-10-19 09:22:00,Burrito Madness,An authentic taste of Mexico.,1,4,cash
2,3,700.0,2025-10-19 09:29:00,Burrito Madness,An authentic taste of Mexico.,1,4,card
3,4,930.0,2025-10-19 09:42:00,Burrito Madness,An authentic taste of Mexico.,1,4,cash
4,5,700.0,2025-10-19 10:11:00,Burrito Madness,An authentic taste of Mexico.,1,4,card
...,...,...,...,...,...,...,...,...
5612,5613,599.0,2025-10-25 16:23:00,SuperSmoothie,"Pick any fruit or vegetable, and we'll make yo...",0,3,cash
5613,5614,499.0,2025-10-25 16:33:00,SuperSmoothie,"Pick any fruit or vegetable, and we'll make yo...",0,3,cash
5614,5615,599.0,2025-10-25 16:39:00,SuperSmoothie,"Pick any fruit or vegetable, and we'll make yo...",0,3,cash
5615,5616,599.0,2025-10-25 16:47:00,SuperSmoothie,"Pick any fruit or vegetable, and we'll make yo...",0,3,cash


## Which truck has the highest number of transactions?

In [26]:
transactions_per_truck = all_data_df.groupby('truck_name').size().reset_index(name='transaction_count')
transactions_per_truck = transactions_per_truck.sort_values(by='transaction_count', ascending=False)   
transactions_per_truck

,truck_name,transaction_count
0,Burrito Madness,1242
1,Cupcakes by Michelle,1205
5,Yoghurt Heaven,1198
3,Kings of Kebabs,892
4,SuperSmoothie,780
2,Hartmann's Jellied Eels,300


## Which truck has the lowest total transaction value?

In [27]:
truck_value = all_data_df.groupby('truck_name')['total'].sum().reset_index(name='total_value')
truck_value = truck_value.sort_values(by='total_value')
truck_value

,truck_name,total_value
2,Hartmann's Jellied Eels,79800.0
4,SuperSmoothie,457220.0
5,Yoghurt Heaven,649463.0
1,Cupcakes by Michelle,714295.0
3,Kings of Kebabs,717862.0
0,Burrito Madness,963219.0


## What is the average transaction value?

In [28]:
average_value = float(all_data_df['total'].mean())
average_value

637.681858643404

## What is the average transaction value for each truck?

In [29]:
average_per_truck = all_data_df.groupby('truck_name')['total'].mean().reset_index(name='average_value')
average_per_truck = average_per_truck.sort_values(by='average_value', ascending=False)
average_per_truck

,truck_name,average_value
3,Kings of Kebabs,804.778027
0,Burrito Madness,775.538647
1,Cupcakes by Michelle,592.775934
4,SuperSmoothie,586.179487
5,Yoghurt Heaven,542.122705
2,Hartmann's Jellied Eels,266.000000


## What proportion of transactions use cash?

In [30]:
payment_proportion_df = all_data_df['payment_method'].value_counts(normalize=True).reset_index()
payment_proportion_df.columns = ['payment_method', 'proportion']
payment_proportion_df

,payment_method,proportion
0,cash,0.536585
1,card,0.463415


## A bar chart showing average transaction total for each truck

In [31]:
bar = alt.Chart(average_per_truck).mark_bar().encode(
    x = alt.X(field = 'truck_name', type = 'nominal', title = 'Truck Name', sort = '-y'),
    y = alt.Y(field = 'average_value', type = 'quantitative', title = 'Average Total')
)

bar

alt.Chart(...)

## A pie chart showing the proportion of transactions made using cash or card

In [32]:
pie = alt.Chart(payment_proportion_df).mark_arc().encode(
    theta = alt.Theta(field = 'proportion', type = 'quantitative'),
    color = alt.Color('payment_method', legend=alt.Legend(title='Payment Method'))
)

pie

alt.Chart(...)

## A line chart showing the total transaction value each day

In [33]:
total_per_day = all_data_df.groupby(all_data_df['at'].dt.floor('D'))['total'].sum().reset_index()
# dt.floor('D) keeps 'at' as a timestamp rounded down to midnight for each day

line = alt.Chart(total_per_day).mark_line().encode(
    x = alt.X(field = 'at', type = 'temporal', title = 'Date'),
    y = alt.Y(field = 'total', type = 'quantitative', title = 'Total Transaction Value')
)

line

alt.Chart(...)